### ספריות

In [ ]:
import os
import sys
import pandas as pd
import geopandas as gpd

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)


### העלת משתנים להרצת הקוד

In [ ]:
path = os.getcwd()

parent = os.path.dirname(path)

software_data_folder_location = os.path.dirname(parent)

df_inputs_outputs = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(software_data_folder_location))

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]

sys.path.append(software_data_folder_location)

## פונקציות

In [ ]:
from py_scripts.adding_students_to_the_population_and_age_distribution import adding_students_to_the_population_and_age_distribution
from py_scripts.adding_yeshiva_students_to_the_population_size import adding_yeshiva_students_to_the_population_size
from py_scripts.quantification_of_employees_in_jtmt_area import quantification_of_employees_in_jtmt_area
from py_scripts.unemployment_calculation import unemployment_calculation
# from py_scripts.calculation_of_days import calculation_of_days
from py_scripts.successive_dispersion import successive_dispersion
from py_scripts.successive_coefficient import successive_coefficient
from py_scripts.layer_splitting_not_follow_traffic_zones import layer_splitting_not_follow_traffic_zones
from py_scripts.distribution_of_employment_branches import distribution_of_employment_branches
from py_scripts.adding_yeshiva_students_to_amount_of_population import adding_yeshiva_students_to_amount_of_population
from py_scripts.adding_palestinian_population import adding_palestinian_population
from py_scripts.export_for_control import export_for_control
from py_scripts.export_in_model_format import export_in_model_format
from py_scripts.logical_controls import logical_controls

### פונקציות גלובליות

In [ ]:
from functions import make_point
from functions import up_load_df
from functions import up_load_gdb
from functions import up_load_shp

## להריץ תלמידים

In [ ]:
%run "C:\Users\dpere\Documents\JTMT\forecast\create_forecast_basic\current\people\students_current_year_230706\students_current_year_230706.ipynb"

## להעלות שכבות

In [ ]:
path=r'{}\background_files\EMP_KIBOLET.gdb'.format(software_data_folder_location)
EMP_kibolet=up_load_gdb(path,'EMP_kibolet')

In [ ]:
EMP_kibolet=EMP_kibolet.fillna(0)

In [ ]:
EMP_kibolet['emp_without_palestin']=EMP_kibolet['kayim_emp']-EMP_kibolet['Palestinians']

In [ ]:
emp_without_palestin_not_okev=EMP_kibolet['emp_without_palestin'].sum().item()

In [ ]:
emp_Education=taz.query('main_secto!="Palestinian"')['emp_Education'].sum().item()

## הוספת סטודנטים לאוכלוסיה ולהתפלגות גילים

In [ ]:
student_dorms=up_load_shp(r'{}\background_files\student_dorms.shp'.format(software_data_folder_location))

In [ ]:
taz=adding_students_to_the_population_and_age_distribution(student_dorms, taz)

## הוספת תלמידי ישיבה (באזורים חרדיים) אל כמות אוכלוסיה,התפלגות גילים 

In [ ]:
taz=adding_yeshiva_students_to_the_population_size(taz)

## כימות מועסקים במרחב צתאל

In [ ]:
taz=quantification_of_employees_in_jtmt_area(taz)

## חישוב אבטלה

In [ ]:
taz=unemployment_calculation(taz)

## חישוב יוממות

In [ ]:
commuting=up_load_shp(r'{}\background_files\commuting_230712.shp'.format(software_data_folder_location))

In [ ]:
taz['commuting']=gpd.sjoin(make_point(taz.reset_index())[['Taz_num','centroid']],commuting[['commuting', 'geometry']]).set_index('Taz_num')['commuting']/100

taz['pop_emp_employed_out_of_jtmt_area']=taz['pop_emp_employed']*taz['commuting']

emp_left_jtmt_area=taz['pop_emp_employed_out_of_jtmt_area'].sum().item()

round(emp_left_jtmt_area,-3)

emp_from_jtmt_area=taz['pop_emp_employed'].sum().item()-emp_left_jtmt_area

# יוממות פנימה מחוץ למרחב
emp_in_jtmt_area=emp_from_jtmt_area*1.07

emp_in_jtmt_area_without_mobile=emp_in_jtmt_area*0.95 #הפחחת עובדים ניידים

round(emp_in_jtmt_area_without_mobile,-3)

emp_okev=emp_in_jtmt_area_without_mobile-emp_Education-emp_without_palestin_not_okev

# taz=calculation_of_days(taz, commuting)

## פיזור עוקב

In [ ]:
taz=successive_dispersion(EMP_kibolet, taz)

## מקדם עוקב

In [ ]:
taz=successive_coefficient(taz, emp_okev, software_data_folder_location)

## פיצול שכבת לא עוקב לאזורי תנועה

In [ ]:
taz=layer_splitting_not_follow_traffic_zones(EMP_kibolet, taz, emp_in_jtmt_area_without_mobile)

## תוצאות

In [ ]:
taz.groupby(by='zonetype')[['total_emp']].sum()#.loc[['Jerusalem','Judea and Samaria']]

In [ ]:
taz.groupby(by='Muni_Heb')[['total_emp']].sum().loc[['ירושלים','בית שמש','ביתר עילית','מודיעין - מכבים - רעות','מעלה אדומים','מודיעין עילית']]

In [ ]:
taz.groupby(by=['Muni_Heb','main_secto'])[['total_emp']].sum().loc[['ירושלים']]

## התפלגות ענפי תעסוקה

In [ ]:
emp_category_type=up_load_shp(r'{}\background_files\emp_category_type.shp'.format(software_data_folder_location)).fillna(0)

In [ ]:
taz=distribution_of_employment_branches(emp_category_type, taz)

## הוספת סטודנטים אל כמות משקי בית

In [ ]:
taz=taz.fillna(0)

In [ ]:
taz['hh']=taz['aprt_20']+taz['student_dorms']/1.5

## הוספת תלמידי ישיבה (באזורים לא חרדיים) אל כמות אוכלוסיה,התפלגות גילים ומשקי בית (לכלל תלמידי הישיבה)

In [ ]:
taz=adding_yeshiva_students_to_amount_of_population(taz)

## הוספת אוכלוסיה פלסטינאית

In [ ]:
path=r'{}\background_files'.format(software_data_folder_location)
taz_demo_pls_2020=up_load_df(path,'taz_demo_pls_2020_and_pre_growth_till_2050')[['Taz_num','pop_2020']].set_index('Taz_num')

In [ ]:
taz=adding_palestinian_population(taz, taz_demo_pls_2020)

## ייצוא לבקרה

In [ ]:
taz=export_for_control(taz)

In [ ]:
col=['Taz_num','Agg_taz_nu','Taz_name','Muni_Heb','jeru_metro',
 'zonetype',
 'in_jerusal','main_secto','aprt_20','student_dorms','student_yeshiva_with_add_from_old','hh','pop_without_dorms_yeshiva',
 'pop', 'hh_size','pop_emp_employed',
 'commuting','student_toddlers',
 'student_gov','cbs_muni_student_left_by_pre_of_demand_left','cbs_muni_student_left_by_pre_of_demand','student_chardi_not_gov',
 'student_arab_not_gov','student','uni_students',
 'student_yeshiva','yeshiva_dorms_pop_15',
 'yeshiva_dorms_pop_20',
 'yeshiva_dorms_pop_25',
 'yeshiva_dorms_pop_sum',
 'add_from_old_student_yeshiva',
 'add_from_kollim_demand', 'emp_from_uni_student','emp_from_Yeshiva_student','emp_from_student',
 'emp_Education','okev',
 'emp_okev',
 'emp_not_okev','total_emp',
 'agri',
 'Indus',
 'Com_hotel',
 'Business',
 'Public']

In [ ]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [ ]:
taz[col].to_excel(r'{}\2020_jtmt_forcast_full_{}.xlsx'.format(forecast_version_folder_location,file_date),index=False)

## ייצוא בפורמט של מודלים

In [ ]:
taz=export_in_model_format(taz)

In [ ]:
col=['Taz_num',
'yosh',
'jeru_metro',
'jerusalem_city',
'main_secto',
'hh',
'pop',
'pop_0',
'pop_5',
'pop_10',
'pop_15',
'pop_20',
'pop_25',
'pop_30',
'pop_35',
'pop_40',
'pop_45',
'pop_50',
'pop_55',
'pop_60',
'pop_65',
'pop_70',
'pop_75up',
'total_emp',
'Indus',
'Com_hotel',
'Business',
'Public',
'emp_Education',
'agri',
'student',
'uni_students',
'student_yeshiva_and_kollim',
'pop_emp_employed']

In [ ]:
col_new_name=['TAZ',
'yosh',
'in_jerusalem_metropolin',
'jerusalem_city',
'sector',
'hh_total',
'pop',
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',
'emp_tot',
'indus',
'com_hotel',
'business',
'public',
'education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed']

In [ ]:
taz=taz[col]

In [ ]:
taz.columns=col_new_name

In [ ]:
taz.to_excel(r'{}\forecast_2020_{}.xlsx'.format(forecast_version_folder_location,file_date),index=False)

## בקרות לוגיות

In [ ]:
taz=logical_controls(taz)